# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [127]:
import pandas as pd
import numpy as np
from scipy import stats

#other packages
import seaborn as sns
import matplotlib.pyplot as plt


In [100]:
data = pd.io.stata.read_stata('data/lakisha_aer.dta')

In [101]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [102]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [104]:
w = data[data.race=='w']
b = data[data.race=='b']

num_w = len(w)
num_b = len(b)
print(num_w, 'number White Names',num_b,"=number Black Names")


(2435, 'number White Names', 2435, '=number Black Names')


In [105]:
num_w_calls=sum(w.call)
num_b_calls=sum(b.call)
num_calls=sum(data.call)
print(num_w_calls,"=White Name callbacks",num_b_calls,"=Black Name callbacks",
      num_calls,'=Number of Calls')


(235.0, '=White Name callbacks', 157.0, '=Black Name callbacks', 392.0, '=Number of Calls')


# Problem: Does race have a significant impact on the rate of callbacks for resumes.

## 1.) What test is appropriate for this problem? Does CLT apply?

The two-proportion z-test would be appropriate. We need the following conditions:

The sampling method for each population is simple random sampling.
The samples are independent.
Each sample includes at least 10 successes and 10 failures. 
Each population is at least 20 times as big as its sample.

We will be testing against two data samples the white sounding names and the black sounding names. A successes is defined as a call back. We assume that the population is US workers 154 million (US Bureau of Labor Statistics 2016). Our data of 4870 entries should be sufficient sample size compared to the population. We have success for both black and white sounding names above 10.

Does Central Limit Theory (CLT) apply? Yes

CLT states: "Regardless of the population distribution model, as the sample size increases, the mean of all samples from the same population will be approximately equal to the mean of the population, given a sufficiently large sample size from a population with a finite level of variance"

The conditions of CLT require the samples must be independent, that the sample size be large, and the variance be finite.
 
Independent condition: Each sample should represent a random sample from the population, or at least follow the population distribution. This is a modeling question here and there is no reason to believe, from the data provided, that the sampling was not random.

Sample size should be large enough. We assume that the population is US workers 154 million (US Bureau of Labor Statistics 2016). Our data of 4870 entries should be sufficient. Usually, a value of n greater than 30 will ensure thta the distribution can be closely approximated bya normal distribution.

## 2.)What are the null and alternate hypotheses?

The null hypothesis: there is no difference between the two population proportions of callbacks for white sounding names compared to callbacks for black sounding names

The alternative-hypothesis: there is a difference between the two population proportions of callbacks for white sounding names compared to callbacks for black sounding names

H0: P1_w = P2_b 
Ha: P1_w ≠ P2_b

P1_w=population proportion one is the white sounding name callbacks
P2_b=populatioa proportion two is the black sounding name callbacks

Assumption: A significance level of 0.05 or 95%


## 3.) Compute margin of error, confidence interval, and p-value. 
Try using both the bootstrapping and the frequentist statistical approaches.

In [106]:
#p1_w is the proportion of callbacks for white sounding names
#p2_b is the proportion of callbacks for black sounding names
p1_w=num_w_calls/num_w
p2_b=num_b_calls/num_b

In [107]:
#calculate pooled sample proportion 

pooled_p= (p1_w * num_w + p2_b* num_b) / (num_w + num_b)
pooled_p

0.080492813141683772

In [108]:
#calculate standard error of the sampling distribution difference between two proportions
std_error= np.sqrt(pooled_p * (1 - pooled_p)* ( (1.0/num_w) + (1.0/num_b) ))
std_error

0.0077968940361704568

In [109]:
#calculate z-score (z)
z = (p1_w - p2_b) / std_error
z

4.1084121524343464

In [110]:
#calculates p value of sample mean compared 
stats.norm.cdf(-z)

1.9919434187925383e-05

In [111]:
#calculates p value of sample mean compared 
1-stats.norm.cdf(z)

1.9919434187887219e-05

we have a two-tailed test, the P-value is the probability that the z-score is less than -4.11 or greater than 4.11.

We use the Normal Distribution to find P(z < -4.11) = 1.9919434187925383e-05, and P(z > 4.11) = 1.9919434187925383e-05. Thus, adding these together the p-value is 3.9838868375850767e-05, extremely small and much less than the rejection level of assumption of significance (0.05).

We reject the null hypothesis. We suspect there is a a differnence in the proportion of callbacks for black sounding names and white sounding names. 


In [112]:
#confidence interval for 95% and margin of error
left_interval_z=(p1_w-p2_b)-(1.96 * std_error)
right_interval_z=(p1_w-p2_b)+(1.96 * std_error)
print"[",left_interval_z,",",right_interval_z,"]", "95% CI"
print((1.96 * std_error),'margin of error',)

[ 0.0167509418986 , 0.0473147665203 ] 95% CI
(0.015281912310894095, 'margin of error')


## 4.)Write a story describing the statistical significance in the context or the original problem.

We are looking at the difference in the proportions of callback for each group, white sounding names and black sounding namnes. We hypothesized that there should not be a difference, which means the difference between the two proportions should be zero. Now the fact that the confidence interval for the difference does not contain zero, we can conclude that there is a statistically significant difference in the two proportion values at the given level of confidence. 

## 5.)Does your analysis mean that race/name is the most important factor in callback success? 
Why or why not? If not, how would you amend your analysis?

No, race may not be the most important factor in callback success. The data actually has many variables that could be influencing the callback rate, education, years of experience, military experience, or job type. We have demonstrated that the sound of a person's name rather white sounding or black could be a contributing factor. We would need to investigate farther to determine if this affect is a causation or just a correlated attribute of callback rates. We would need to test again with observations, resumes, that had all variables except different black sounding names and white sounding names to see if there was still a difference in the proportion of callbacks. This would then be a strong indication of race/name being a causation affect on callback rates.  



<div class="span5 alert alert-success">
<h2>SOLUTION: What test is appropriate for this problem?</h2>

<p>This is an interesting question, and it can be argued that there are two possible correct answers (such is life). First note the wording of the question: does race have a *significant impact* on callback. This usually suggests a $\chi^2$-test, but the $\chi^2$-test requires *frequencies* rather than percentages. Interestingly, one of the code snippets above uses frequency so we will look at that in a bit.</p>

For now, a two-sample $z$-test:

$$z = \frac{\left( \hat{p}_W - \hat{p}_B \right) - 0}{\sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}}$$

where

$$\hat{p} = \frac{y_W + y_B}{n_W + n_B}$$

The null and alternate hypotheses:

$$H_0: p_B = p_W$$
$$H_A: p_B \ne p_W$$

CLT applies because we assume that the samples are representative of the population. The observations in each sample are assumed to be independent since the sample was drawn randomly.

In [92]:
#solution code
w = data[data.race=='w']
b = data[data.race=='b']

n_w = len(w)
n_b = len(b)

prop_w = np.sum(w.call) / len(w)
prop_b = np.sum(b.call) / len(b)

prop_diff = prop_w - prop_b
phat = (np.sum(w.call) + np.sum(b.call)) / (len(w) + len(b))

z = prop_diff / np.sqrt(phat * (1 - phat) * ((1.0 / n_w) + (10 / n_b)))
pval = stats.norm.cdf(-z) * 2
print("Z score: {}".format(z))
print("P-value: {}".format(pval))

Z score: 5.81017218579
P-value: 6.24086216103e-09


In [94]:
moe = 1.96 * np.sqrt(phat * (1 - phat) * ((1.0 / n_w) + (1.0 / n_b)))
ci = prop_diff + np.array([-1, 1]) * moe
print("Margin of Error: {}".format(moe))
print("Confidence interval: {}".format(ci))

Margin of Error: 0.0152819123109
Confidence interval: [ 0.01675094  0.04731477]


<div class="span5 alert alert-success">
<p>The p-value is practically 0 so we reject the null hypothesis that white and black sounding names have the same callback rate. They do not.</p>

<p>Since 0 is not in the confidence interval, we reject the null hypothesis with the same conclusion.</p>

<h4>SOLUTION: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</h4>

<p>No! While our test did show that there is a difference in callback rate based on race alone, there are other variables that may also contribute to, or interact with, race to explain the difference. In the original research paper, the researchers cited geography/city as a confounding variable. Additionally, we could also look at education and experience levels as well. But, in our very narrow example, we have shown that there is a significant difference in callback rates between white people and black people.</p>

<p>The $\chi^2$ approach:</p>
</div>

In [12]:
cont_table = pd.crosstab(index=data.call, columns=data.race)
chi2, pval, _, _ = stats.chi2_contingency(cont_table)
print("Chi-squared test statistic: {}".format(chi2))
print("p-value: {}".format(pval))

Chi-squared test statistic: 16.4490285842
p-value: 4.99757838996e-05


<div class="span5 alert alert-success">
The chi-squared test yields a similar result. We reject the null hypothesis that race and callback rate are independent. The margin of error and confidence interval calculations are a bit more complicated because the chi-squared distribution is not always symmetric, depending on the number of degrees of freedom.
</div>